In [1]:
%pip install openai numpy matplotlib pydantic --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


# Simple Agentic Loop Example with the Responses API

This notebook demonstrates how to build a simple agent using the OpenAI Responses API with an agentic loop.

We create **two agents**:

1. **Simple Agent:**
   - The agent enters a loop where it sends a request and checks whether the response contains a tool call. 
   - If so, it executes the function (e.g. _get_weather_) and appends the result to the conversation.
   - The loop stops when there are no more tool calls and the response contains final text (`output_text`).

2. **Objective-Based Agent:**
   - The agent is given a custom objective function (e.g. check whether the phrase "task complete" is in the output).
   - It loops until the objective function returns `True`.

Below, you'll see the code for each agent along with explanations.

## Prerequisites

Make sure you have the OpenAI Python package installed and have set up authentication with your API key. Also, you should have any supporting code (for example, a real implementation of `get_weather`) ready.

For demonstration, we'll define a simple `get_weather` function which calls a public weather API.

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
import requests

In [3]:
MODEL = "gpt-4.1-nano"

In [4]:
# Load environment variables from .env file
load_dotenv(override=True)  # Ensure .env values are loaded and override existing ones

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment. Please check your .env file.")

# Create an OpenAI client instance
client = OpenAI(
    api_key=api_key
)

In [5]:
def get_weather(latitude, longitude):
    # For demonstration we use a public weather API
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
    )
    data = response.json()
    return data['current']['temperature_2m']



# Define the get_weather tool with a strict JSON schema
weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": { "type": "number", "description": "Latitude of the location." },
            "longitude": { "type": "number", "description": "Longitude of the location." }
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}

# We can also define more tools such as send_email or search_knowledge_base if desired.
# For the purpose of these examples, we will use just the weather tool.

tools = [weather_tool]

## Agent 1: Simple Agent Loop

This agent sends a prompt (asking about the weather), then enters an agentic loop. 

At each turn, it calls the Responses API:

- **If the response contains a tool call:** The agent executes the function (using our `get_weather` tool) and appends the function result to the conversation as a new message.
- **If the response provides output text:** The agent stops, printing the final output.

Below is the code for the simple agent.

In [16]:
# Initialize the conversation with a user prompt
messages = [{"role": "developer", "content": "What's the weather like in Paris today?"}]

while True:
    response = client.responses.create(
        model=MODEL,
        input=messages,
        tools=tools
    )
    
    # Process all function calls in the response
    if response.output:
        for output_item in response.output:
            if hasattr(output_item, 'type') and output_item.type == "function_call":
                # Append the function call to the messages:
                messages.append(output_item)

                tool_call = output_item
                args = json.loads(tool_call.arguments)
                
                # Execute the function, e.g. get_weather (simulate using our get_weather function)
                result = get_weather(args['latitude'], args['longitude'])
                print(f"Executed {tool_call.name}: Result = {result}°C")

                # Append the function call output to the conversation
                messages.append({
                    "type": "function_call_output",
                    "call_id": tool_call.call_id,
                    "output": str(result)
                })
    
    # If the final output text is provided, break the loop
    if hasattr(response, 'output_text') and response.output_text:
        print("Final Agent Output:", response.output_text)
        break

    # Otherwise, continue the loop (in a full implementation, you might update the conversation further)
    
    # For simplicity, break if no further tool calls are made
    if not response.output:
        break

Executed get_weather: Result = 13.2°C
Final Agent Output: The current temperature in Paris today is approximately 13.2°C. If you need more detailed weather information, such as conditions or forecast, let me know!


In [17]:
def agent_loop(messages, tools):
    while True:
        response = client.responses.create(
            model=MODEL,
            input=messages,
            tools=tools
        )
        
        # Process all function calls in the response
        if response.output:
            for output_item in response.output:
                if hasattr(output_item, 'type') and output_item.type == "function_call":
                    # Append the function call to the messages:
                    messages.append(output_item)

                    tool_call = output_item
                    args = json.loads(tool_call.arguments)
                    
                    # Execute the function, e.g. get_weather (simulate using our get_weather function)
                    result = get_weather(args['latitude'], args['longitude'])
                    print(f"Executed {tool_call.name}: Result = {result}°C")

                    # Append the function call output to the conversation
                    messages.append({
                        "type": "function_call_output",
                        "call_id": tool_call.call_id,
                        "output": str(result)
                    })
        
        # If the final output text is provided, break the loop
        if hasattr(response, 'output_text') and response.output_text:
            print("Final Agent Output:", response.output_text)
            break

        # Otherwise, continue the loop (in a full implementation, you might update the conversation further)
        
        # For simplicity, break if no further tool calls are made
        if not response.output:
            break

In [18]:
# Initialize the conversation with a user prompt
messages = [{"role": "developer", "content": "What's the weather like in Paris today? Before replying I want you to also get the weather for Berlin."}]

In [19]:
agent_loop(messages=messages, tools=tools)

Executed get_weather: Result = 13.2°C
Executed get_weather: Result = 23.8°C
Final Agent Output: Today, the weather in Paris is 13.2°C, while in Berlin it is warmer at 23.8°C.


---

## Agent 2: Agent with Custom Objective Function

This agent uses a custom objective function to decide whether to continue looping. In this example, the objective function checks if the agent's output text contains the phrase "task complete".

The agent will continue to request responses (and execute any tool calls) until the objective is met. Note that this is a simplified demonstration intended for teaching purposes.

In [6]:
import json

def objective_met(search_count, max_searches=5):
    # Stop once we've searched more than max_searches cities
    return search_count > max_searches

# Initial conversation: developer sets the task, user gives the first city
messages= [
    {
        "role": "developer",
        "content": (
            "Your goal is to gather weather for at least 5 different cities. "
            "Once you've done that, respond with 'task complete'."
        )
    },
    {"role": "user", "content": "Search the weather in Berlin."}
]

search_count = 0

while True:
    response = client.responses.create(
        model=MODEL,
        input=messages,
        tools=tools
    )
    
    # 1) Handle any function calls (e.g. get_weather)
    for item in response.output or []:
        if getattr(item, 'type', None) == "function_call":
            messages.append(item)  # pass the function call back into context
            
            # parse and execute
            args = json.loads(item.arguments)
            temp = get_weather(args['latitude'], args['longitude'])
            print(f"Executed {item.name}: {temp}°C")
            
            # append function result
            messages.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": str(temp)
            })
            
            search_count += 1
    
    # 2) Check for assistant output_text
    if hasattr(response, 'output_text') and response.output_text:
        text = response.output_text
        print("Agent says:", text)
        messages.append({"role": "assistant", "content": text})
    
    # 3) Check objective
    if objective_met(search_count):
        print(f"Objective met: searched {search_count} cities. Task complete.")
        break
    
    # 4) If not done, prompt for the next city
    messages.append({
        "role": "user",
        "content": f"We've searched {search_count} so far. Please search another city."
    })


Executed get_weather: 27.8°C
Executed get_weather: 26.0°C
Executed get_weather: 22.9°C


KeyboardInterrupt: 

In [28]:
print(messages[-1])

{'type': 'function_call_output', 'call_id': 'call_ktvU8fMFC6uilq92QA2bKeYX', 'output': '15.4'}


## Summary

- **Agent 1 (Simple Agent):**
  - Uses a while loop to repeatedly call the Responses API.
  - Checks for tool (function) calls, executes them, and appends the output to the conversation.
  - The loop stops when a final response (output_text) is provided and there are no more tool calls.

- **Agent 2 (Objective-Based Agent):**
  - Uses a custom objective function (`objective_met`) to decide when to stop the loop.
  - Continues to gather responses and execute tools until the agent's output includes a key phrase ("task complete").

Both agents illustrate how you can build agentic loops from scratch using the Responses API and custom tools. Adapt and extend these examples as needed for more complex tasks!